## Libs import


In [0]:
from __future__ import division

import scipy
import numpy as np
import pandas as pd

from scipy import stats
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.weightstats import CompareMeans, DescrStatsW, ztest

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Task 1


in one of the episodes of the "Myth Busters" series, they tested contagious yawns myth. The experiment was attended by 50 subjects who were interviewed on the program. Each of them spoke with a recruiter; at the end of 34 out of 50 conversations, the recruiter yawned. Then the subjects were asked to wait for the recruiter's decision in the next empty room. While waiting, 10 out of 34 subjects in the experimental group and 4 out of 16 subjects in the control group began to yawn. Thus, the difference in the proportion of yawning people in these two groups is approximately 4.4%. Leading concluded that confirmed. Are statistical values ​​statistical? Calculate the achievable significance level with alternate infectivity, up to ten digits after the decimal point.

In [0]:
data_exp = np.array([1 if i < 10 else 0 for i in range(34)])
data_ctrl = np.array([1 if i < 4 else 0 for i in range(16)])

In [3]:
print(f'Mean experimental value: {data_exp.mean():.4f}')
print(f'Mean control value: {data_ctrl.mean():.4f}') 

Mean experimental value: 0.2941
Mean control value: 0.2500


In [0]:
conf_interval_data_exp = proportion_confint(sum(data_exp), 
                                            len(data_exp),
                                            method = 'wilson')

conf_interval_data_ctrl = proportion_confint(sum(data_ctrl), 
                                            len(data_ctrl),
                                            method = 'wilson')

In [5]:
print(f'95%% confidence interval for exp group: [{conf_interval_data_exp[0]:.4f}, {conf_interval_data_exp[1]:.4f}]')
print(f'95%% confidence interval for a ctrl group: [{conf_interval_data_ctrl[0]:.4f}, {conf_interval_data_ctrl[1]:.4f}]')

95%% confidence interval for exp group: [0.1683, 0.4617]
95%% confidence interval for a ctrl group: [0.1018, 0.4950]


### Z-test for independant proportions

In [0]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [0]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2) / len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [0]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [9]:
l_b, r_b = proportions_diff_confint_ind(data_exp, data_ctrl)
print(f"95%% confidence interval for a difference between proportions: [{l_b:.4f}, {r_b:.4f}]")

95%% confidence interval for a difference between proportions: [-0.2176, 0.3058]


In [10]:
print (f"p-value: {proportions_diff_z_test(proportions_diff_z_stat_ind(data_exp, data_ctrl)):.4f}")

p-value: 0.7459


In [11]:
print (f"p-value: {proportions_diff_z_test(proportions_diff_z_stat_ind(data_exp, data_ctrl), 'greater'):.4f}")

p-value: 0.3729


## Task 2


There are measurement data of two hundred Swiss thousand-franc banknotes that were in circulation in the first half of the XX century. One hundred of the banknotes were real and one hundred were fake. The figure below shows the measured signs:

![alt text](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/GN7XXyT6EeaNKg4a0RSP6Q_5d67a093a6c9bca6e4d89fc9680fbf89_CHF1000_2_back_horizontal.jpg?expiry=1567209600000&hmac=zSpB8CAaUfgkOv5ullBxeReqJkm2f92miZtBtaOFzHQ)

### Data 


In [0]:
banknotes_data = pd.read_csv('https://raw.githubusercontent.com/OzmundSedler/100-Days-Of-ML-Code/master/week_11/datasets/banknotes.txt', sep='\t')

In [13]:
banknotes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
X1      200 non-null float64
X2      200 non-null float64
X3      200 non-null float64
X4      200 non-null float64
X5      200 non-null float64
X6      200 non-null float64
real    200 non-null int64
dtypes: float64(6), int64(1)
memory usage: 11.0 KB


In [14]:
banknotes_data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [15]:
banknotes_data.describe()

,X1,X2,X3,X4,X5,X6,real
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,214.896000,130.121500,129.956500,9.417500,10.650500,140.483500,0.500000
std,0.376554,0.361026,0.404072,1.444603,0.802947,1.152266,0.501255
min,213.800000,129.000000,129.000000,7.200000,7.700000,137.800000,0.000000
25%,214.600000,129.900000,129.700000,8.200000,10.100000,139.500000,0.000000
50%,214.900000,130.200000,130.000000,9.100000,10.600000,140.450000,0.500000
75%,215.100000,130.400000,130.225000,10.600000,11.200000,141.500000,1.000000
max,216.300000,131.000000,131.100000,12.700000,12.300000,142.400000,1.000000


### Task 1.1


Separate 50 random observations into a test sample using the sklearn.cross_validation.train_test_split function (fix random state = 1). On the remaining 150, configure two banknote counterfeit classifiers:

logistic regression according to features X1, X2, X3; logistic regression according to features X4, X5, X6. For each of the classifiers, make class label predictions on the test set. Are the fractions of erroneous predictions of the two classifiers the same? Test the hypothesis, calculate the achieved significance level. Enter the number of the first significant digit (for example, if you received 5.5 × 10−8, you need to enter 8).

In [0]:
banknotes_train, banknotes_test = train_test_split(
    banknotes_data,
    test_size=50,
    random_state=1,
)

In [0]:
features_1 = ['X1', 'X2', 'X3']
features_2 = ['X2', 'X3', 'X4']

In [18]:
clf_logreg_1 = LogisticRegression()
clf_logreg_1.fit(banknotes_train[features_1].values,
                 banknotes_train['real'].values)

clf_logreg_2 = LogisticRegression()
clf_logreg_2.fit(banknotes_train[features_2].values,
                banknotes_train['real'].values)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
pred_1 = clf_logreg_1.predict(banknotes_test[features_1].values)
print('Error rate pred1: %f' % (1 - accuracy_score(banknotes_test['real'].values, pred_1)))
err_1 = np.array( [1 if banknotes_test['real'].values[i] == pred_1[i] else 0 for i in range(len(pred_1))] )

Error rate pred1: 0.200000


In [20]:
pred_2 = clf_logreg_2.predict(banknotes_test[features_2].values)
print('Error rate pred2: %f' % (1 - accuracy_score(banknotes_test['real'].values, pred_2)))
err_2 = np.array( [1 if banknotes_test['real'].values[i] == pred_2[i] else 0 for i in range(len(pred_2))] )

Error rate pred2: 0.140000


### Task 2

Calculate the 95% confidence interval for the difference in the error shares of the two classifiers. What is its border closest to zero?


In [0]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [0]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [23]:
l_r, r_b = proportions_diff_confint_rel(err_1, err_2)
print(f'95%% confidence interval for a difference between predictions: [{l_r:.4f}, {r_b:.4f}]')
      

95%% confidence interval for a difference between predictions: [-0.2004, 0.0804]


In [24]:
print(f'p-value: {proportions_diff_z_test(proportions_diff_z_stat_rel(err_1, err_2)):.4f}')

p-value: 0.4021


## Task 3

Each year, more than 200,000 people around the world pass the standardized GMAT exam when they enroll in MBA programs. The average result is 525 points, the standard deviation is 100 points.

One hundred students completed special preparatory courses and passed the exam. The average score they received is 541.4. Test the hypothesis of program inefficiency versus the one-way alternative that the program works. Is the null hypothesis rejected at significance level 0.05? Enter the significance level rounded to 4 digits after the decimal point.

In [0]:
mean_gen = 525
num_gen = 200000
sigma = 100

num_sample = 100
mean_sample = 541.4

In [0]:
def z_test(mean_val, exp_val, st_dev, num):
    standard_error = st_dev / np.sqrt(num)
    return (mean_val - exp_val) / standard_error

In [0]:
def p_val_greater(z_stat):
    return 1 - stats.norm.cdf(z_stat)

In [28]:
z_stat = z_test(mean_sample, mean_gen, sigma, num_sample)
print(z_stat)

1.6399999999999977


In [29]:
p_val = p_val_greater(abs(z_stat))
print(p_val)

0.05050258347410397


Evaluate now the effectiveness of preparatory courses, the average score of 100 graduates of which is 541.5. Is the same null hypothesis against the same alternative rejected at significance level 0.05? Enter the significance level rounded to 4 digits after the decimal point.

In [0]:
avg_mark_2 = 541.5

In [31]:
z_stat2 = z_test(avg_mark_2, mean_gen, sigma, num_sample)
print(z_stat2)

1.65


In [32]:
p_val = p_val_greater(abs(z_stat2))
print(p_val)

0.0494714680336481
